In [1]:
import math

import sys
sys.path.insert(0, '..')
#import d2l
import torch
import torch.nn as nn
import torch.optim as optim
import time
import numpy
import yacht_main as yacht

In [2]:
INPUT_SIZE = 6
OUTPUT_SIZE = 12

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(INPUT_SIZE)
    
net = torch.nn.Sequential(
    Reshape(),
    
    nn.Linear(in_features=INPUT_SIZE, out_features=50),
    nn.LeakyReLU(),
    nn.Linear(50,30),
    nn.LeakyReLU(),
    nn.Linear(30,OUTPUT_SIZE)
    
    #nn.Linear(in_features=INPUT_SIZE, out_features = OUTPUT_SIZE)
)

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.uniform_(m.weight,-0.5,0.5)

net = net.float()
net.apply(init_weights)

Sequential(
  (0): Reshape()
  (1): Linear(in_features=6, out_features=50, bias=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=50, out_features=30, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Linear(in_features=30, out_features=12, bias=True)
)

In [3]:
def try_gpu():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

device = try_gpu()
print(torch.cuda.is_available())
net = net.to(device)

True


In [4]:
train_data = []
SCALE = 30.0


def pack(org):
    return (org/SCALE)

def unpack(norm):
    return round(SCALE*float(norm))

for d1 in range(6):
    for d2 in range(6):
        for d3 in range(6):
            for d4 in range(6):
                for d5 in range(6):
                    dices = [d1+1,d2+1,d3+1,d4+1,d5+1]
                    scores = [pack(f(dices)) for f in yacht.score_func]
                    #scores = [yacht.score_func[6](dices)/SCALE]
                    
                    dt = torch.tensor([dices.count(i+1) for i in range(6)], dtype = torch.float32, requires_grad = False)
                    st = torch.tensor(scores, dtype = torch.float32, requires_grad = False)
                    
                    train_data.append((dt,st))

print(str(len(train_data)))

for x,y in train_data:
    cont = ""
    for a in x:
        cont += str(int(a))+" "
    cont += " | "
    for b in y:
        cont += str(unpack(b))+" "
    print(cont)


7776
5 0 0 0 0 0  | 5 0 0 0 0 0 5 5 5 0 0 50 
4 1 0 0 0 0  | 4 2 0 0 0 0 6 6 0 0 0 0 
4 0 1 0 0 0  | 4 0 3 0 0 0 7 7 0 0 0 0 
4 0 0 1 0 0  | 4 0 0 4 0 0 8 8 0 0 0 0 
4 0 0 0 1 0  | 4 0 0 0 5 0 9 9 0 0 0 0 
4 0 0 0 0 1  | 4 0 0 0 0 6 10 10 0 0 0 0 
4 1 0 0 0 0  | 4 2 0 0 0 0 6 6 0 0 0 0 
3 2 0 0 0 0  | 3 4 0 0 0 0 7 0 7 0 0 0 
3 1 1 0 0 0  | 3 2 3 0 0 0 8 0 0 0 0 0 
3 1 0 1 0 0  | 3 2 0 4 0 0 9 0 0 0 0 0 
3 1 0 0 1 0  | 3 2 0 0 5 0 10 0 0 0 0 0 
3 1 0 0 0 1  | 3 2 0 0 0 6 11 0 0 0 0 0 
4 0 1 0 0 0  | 4 0 3 0 0 0 7 7 0 0 0 0 
3 1 1 0 0 0  | 3 2 3 0 0 0 8 0 0 0 0 0 
3 0 2 0 0 0  | 3 0 6 0 0 0 9 0 9 0 0 0 
3 0 1 1 0 0  | 3 0 3 4 0 0 10 0 0 0 0 0 
3 0 1 0 1 0  | 3 0 3 0 5 0 11 0 0 0 0 0 
3 0 1 0 0 1  | 3 0 3 0 0 6 12 0 0 0 0 0 
4 0 0 1 0 0  | 4 0 0 4 0 0 8 8 0 0 0 0 
3 1 0 1 0 0  | 3 2 0 4 0 0 9 0 0 0 0 0 
3 0 1 1 0 0  | 3 0 3 4 0 0 10 0 0 0 0 0 
3 0 0 2 0 0  | 3 0 0 8 0 0 11 0 11 0 0 0 
3 0 0 1 1 0  | 3 0 0 4 5 0 12 0 0 0 0 0 
3 0 0 1 0 1  | 3 0 0 4 0 6 13 0 0 0 0 0 
4 0 0 0 1 0  | 4 0 0 0

2 0 0 1 2 0  | 2 0 0 4 10 0 16 0 0 0 0 0 
2 0 0 1 1 1  | 2 0 0 4 5 6 17 0 0 0 0 0 
2 1 0 1 1 0  | 2 2 0 4 5 0 13 0 0 0 0 0 
1 2 0 1 1 0  | 1 4 0 4 5 0 14 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 0 
1 1 0 2 1 0  | 1 2 0 8 5 0 16 0 0 0 0 0 
1 1 0 1 2 0  | 1 2 0 4 10 0 17 0 0 0 0 0 
1 1 0 1 1 1  | 1 2 0 4 5 6 18 0 0 0 0 0 
2 0 1 1 1 0  | 2 0 3 4 5 0 14 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 0 
1 0 2 1 1 0  | 1 0 6 4 5 0 16 0 0 0 0 0 
1 0 1 2 1 0  | 1 0 3 8 5 0 17 0 0 0 0 0 
1 0 1 1 2 0  | 1 0 3 4 10 0 18 0 0 0 0 0 
1 0 1 1 1 1  | 1 0 3 4 5 6 19 0 0 0 0 0 
2 0 0 2 1 0  | 2 0 0 8 5 0 15 0 0 0 0 0 
1 1 0 2 1 0  | 1 2 0 8 5 0 16 0 0 0 0 0 
1 0 1 2 1 0  | 1 0 3 8 5 0 17 0 0 0 0 0 
1 0 0 3 1 0  | 1 0 0 12 5 0 18 0 0 0 0 0 
1 0 0 2 2 0  | 1 0 0 8 10 0 19 0 0 0 0 0 
1 0 0 2 1 1  | 1 0 0 8 5 6 20 0 0 0 0 0 
2 0 0 1 2 0  | 2 0 0 4 10 0 16 0 0 0 0 0 
1 1 0 1 2 0  | 1 2 0 4 10 0 17 0 0 0 0 0 
1 0 1 1 2 0  | 1 0 3 4 10 0 18 0 0 0 0 0 
1 0 0 2 2 0  | 1 0 0 8 10 0 19 0 0 0 0 0 
1 0

0 2 2 0 0 1  | 0 4 6 0 0 6 16 0 0 0 0 0 
0 2 1 1 0 1  | 0 4 3 4 0 6 17 0 0 0 0 0 
0 2 1 0 1 1  | 0 4 3 0 5 6 18 0 0 0 0 0 
0 2 1 0 0 2  | 0 4 3 0 0 12 19 0 0 0 0 0 
2 1 2 0 0 0  | 2 2 6 0 0 0 10 0 0 0 0 0 
1 2 2 0 0 0  | 1 4 6 0 0 0 11 0 0 0 0 0 
1 1 3 0 0 0  | 1 2 9 0 0 0 12 0 0 0 0 0 
1 1 2 1 0 0  | 1 2 6 4 0 0 13 0 0 15 0 0 
1 1 2 0 1 0  | 1 2 6 0 5 0 14 0 0 0 0 0 
1 1 2 0 0 1  | 1 2 6 0 0 6 15 0 0 0 0 0 
1 2 2 0 0 0  | 1 4 6 0 0 0 11 0 0 0 0 0 
0 3 2 0 0 0  | 0 6 6 0 0 0 12 0 12 0 0 0 
0 2 3 0 0 0  | 0 4 9 0 0 0 13 0 13 0 0 0 
0 2 2 1 0 0  | 0 4 6 4 0 0 14 0 0 0 0 0 
0 2 2 0 1 0  | 0 4 6 0 5 0 15 0 0 0 0 0 
0 2 2 0 0 1  | 0 4 6 0 0 6 16 0 0 0 0 0 
1 1 3 0 0 0  | 1 2 9 0 0 0 12 0 0 0 0 0 
0 2 3 0 0 0  | 0 4 9 0 0 0 13 0 13 0 0 0 
0 1 4 0 0 0  | 0 2 12 0 0 0 14 14 0 0 0 0 
0 1 3 1 0 0  | 0 2 9 4 0 0 15 0 0 0 0 0 
0 1 3 0 1 0  | 0 2 9 0 5 0 16 0 0 0 0 0 
0 1 3 0 0 1  | 0 2 9 0 0 6 17 0 0 0 0 0 
1 1 2 1 0 0  | 1 2 6 4 0 0 13 0 0 15 0 0 
0 2 2 1 0 0  | 0 4 6 4 0 0 14 0 0 0 0 0 
0 1 3 1 

0 1 1 2 0 1  | 0 2 3 8 0 6 19 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 0 
0 2 1 1 1 0  | 0 4 3 4 5 0 16 0 0 15 0 0 
0 1 2 1 1 0  | 0 2 6 4 5 0 17 0 0 15 0 0 
0 1 1 2 1 0  | 0 2 3 8 5 0 18 0 0 15 0 0 
0 1 1 1 2 0  | 0 2 3 4 10 0 19 0 0 15 0 0 
0 1 1 1 1 1  | 0 2 3 4 5 6 20 0 0 15 30 0 
1 1 1 1 0 1  | 1 2 3 4 0 6 16 0 0 15 0 0 
0 2 1 1 0 1  | 0 4 3 4 0 6 17 0 0 0 0 0 
0 1 2 1 0 1  | 0 2 6 4 0 6 18 0 0 0 0 0 
0 1 1 2 0 1  | 0 2 3 8 0 6 19 0 0 0 0 0 
0 1 1 1 1 1  | 0 2 3 4 5 6 20 0 0 15 30 0 
0 1 1 1 0 2  | 0 2 3 4 0 12 21 0 0 0 0 0 
2 1 1 0 1 0  | 2 2 3 0 5 0 12 0 0 0 0 0 
1 2 1 0 1 0  | 1 4 3 0 5 0 13 0 0 0 0 0 
1 1 2 0 1 0  | 1 2 6 0 5 0 14 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 0 
1 1 1 0 2 0  | 1 2 3 0 10 0 16 0 0 0 0 0 
1 1 1 0 1 1  | 1 2 3 0 5 6 17 0 0 0 0 0 
1 2 1 0 1 0  | 1 4 3 0 5 0 13 0 0 0 0 0 
0 3 1 0 1 0  | 0 6 3 0 5 0 14 0 0 0 0 0 
0 2 2 0 1 0  | 0 4 6 0 5 0 15 0 0 0 0 0 
0 2 1 1 1 0  | 0 4 3 4 5 0 16 0 0 15 0 0 
0 2 1 0 2 0  | 0 4 3 0 10 0 17 0 0 0 0 0

2 1 0 1 1 0  | 2 2 0 4 5 0 13 0 0 0 0 0 
1 2 0 1 1 0  | 1 4 0 4 5 0 14 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 0 
1 1 0 2 1 0  | 1 2 0 8 5 0 16 0 0 0 0 0 
1 1 0 1 2 0  | 1 2 0 4 10 0 17 0 0 0 0 0 
1 1 0 1 1 1  | 1 2 0 4 5 6 18 0 0 0 0 0 
2 0 1 1 1 0  | 2 0 3 4 5 0 14 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 0 
1 0 2 1 1 0  | 1 0 6 4 5 0 16 0 0 0 0 0 
1 0 1 2 1 0  | 1 0 3 8 5 0 17 0 0 0 0 0 
1 0 1 1 2 0  | 1 0 3 4 10 0 18 0 0 0 0 0 
1 0 1 1 1 1  | 1 0 3 4 5 6 19 0 0 0 0 0 
2 0 0 2 1 0  | 2 0 0 8 5 0 15 0 0 0 0 0 
1 1 0 2 1 0  | 1 2 0 8 5 0 16 0 0 0 0 0 
1 0 1 2 1 0  | 1 0 3 8 5 0 17 0 0 0 0 0 
1 0 0 3 1 0  | 1 0 0 12 5 0 18 0 0 0 0 0 
1 0 0 2 2 0  | 1 0 0 8 10 0 19 0 0 0 0 0 
1 0 0 2 1 1  | 1 0 0 8 5 6 20 0 0 0 0 0 
2 0 0 1 2 0  | 2 0 0 4 10 0 16 0 0 0 0 0 
1 1 0 1 2 0  | 1 2 0 4 10 0 17 0 0 0 0 0 
1 0 1 1 2 0  | 1 0 3 4 10 0 18 0 0 0 0 0 
1 0 0 2 2 0  | 1 0 0 8 10 0 19 0 0 0 0 0 
1 0 0 1 3 0  | 1 0 0 4 15 0 20 0 0 0 0 0 
1 0 0 1 2 1  | 1 0 0 4 10 6 21 0 0 0 0 0 
2 

0 2 0 2 1 0  | 0 4 0 8 5 0 17 0 0 0 0 0 
0 1 1 2 1 0  | 0 2 3 8 5 0 18 0 0 15 0 0 
0 1 0 3 1 0  | 0 2 0 12 5 0 19 0 0 0 0 0 
0 1 0 2 2 0  | 0 2 0 8 10 0 20 0 0 0 0 0 
0 1 0 2 1 1  | 0 2 0 8 5 6 21 0 0 0 0 0 
1 1 0 1 2 0  | 1 2 0 4 10 0 17 0 0 0 0 0 
0 2 0 1 2 0  | 0 4 0 4 10 0 18 0 0 0 0 0 
0 1 1 1 2 0  | 0 2 3 4 10 0 19 0 0 15 0 0 
0 1 0 2 2 0  | 0 2 0 8 10 0 20 0 0 0 0 0 
0 1 0 1 3 0  | 0 2 0 4 15 0 21 0 0 0 0 0 
0 1 0 1 2 1  | 0 2 0 4 10 6 22 0 0 0 0 0 
1 1 0 1 1 1  | 1 2 0 4 5 6 18 0 0 0 0 0 
0 2 0 1 1 1  | 0 4 0 4 5 6 19 0 0 0 0 0 
0 1 1 1 1 1  | 0 2 3 4 5 6 20 0 0 15 30 0 
0 1 0 2 1 1  | 0 2 0 8 5 6 21 0 0 0 0 0 
0 1 0 1 2 1  | 0 2 0 4 10 6 22 0 0 0 0 0 
0 1 0 1 1 2  | 0 2 0 4 5 12 23 0 0 0 0 0 
2 0 1 1 1 0  | 2 0 3 4 5 0 14 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 0 
1 0 2 1 1 0  | 1 0 6 4 5 0 16 0 0 0 0 0 
1 0 1 2 1 0  | 1 0 3 8 5 0 17 0 0 0 0 0 
1 0 1 1 2 0  | 1 0 3 4 10 0 18 0 0 0 0 0 
1 0 1 1 1 1  | 1 0 3 4 5 6 19 0 0 0 0 0 
1 1 1 1 1 0  | 1 2 3 4 5 0 15 0 0 15 30 

1 0 0 2 2 0  | 1 0 0 8 10 0 19 0 0 0 0 0 
0 1 0 2 2 0  | 0 2 0 8 10 0 20 0 0 0 0 0 
0 0 1 2 2 0  | 0 0 3 8 10 0 21 0 0 0 0 0 
0 0 0 3 2 0  | 0 0 0 12 10 0 22 0 22 0 0 0 
0 0 0 2 3 0  | 0 0 0 8 15 0 23 0 23 0 0 0 
0 0 0 2 2 1  | 0 0 0 8 10 6 24 0 0 0 0 0 
1 0 0 2 1 1  | 1 0 0 8 5 6 20 0 0 0 0 0 
0 1 0 2 1 1  | 0 2 0 8 5 6 21 0 0 0 0 0 
0 0 1 2 1 1  | 0 0 3 8 5 6 22 0 0 15 0 0 
0 0 0 3 1 1  | 0 0 0 12 5 6 23 0 0 0 0 0 
0 0 0 2 2 1  | 0 0 0 8 10 6 24 0 0 0 0 0 
0 0 0 2 1 2  | 0 0 0 8 5 12 25 0 0 0 0 0 
2 0 0 1 2 0  | 2 0 0 4 10 0 16 0 0 0 0 0 
1 1 0 1 2 0  | 1 2 0 4 10 0 17 0 0 0 0 0 
1 0 1 1 2 0  | 1 0 3 4 10 0 18 0 0 0 0 0 
1 0 0 2 2 0  | 1 0 0 8 10 0 19 0 0 0 0 0 
1 0 0 1 3 0  | 1 0 0 4 15 0 20 0 0 0 0 0 
1 0 0 1 2 1  | 1 0 0 4 10 6 21 0 0 0 0 0 
1 1 0 1 2 0  | 1 2 0 4 10 0 17 0 0 0 0 0 
0 2 0 1 2 0  | 0 4 0 4 10 0 18 0 0 0 0 0 
0 1 1 1 2 0  | 0 2 3 4 10 0 19 0 0 15 0 0 
0 1 0 2 2 0  | 0 2 0 8 10 0 20 0 0 0 0 0 
0 1 0 1 3 0  | 0 2 0 4 15 0 21 0 0 0 0 0 
0 1 0 1 2 1  | 0 2 0 4 10 6 22 0

1 0 1 0 1 2  | 1 0 3 0 5 12 21 0 0 0 0 0 
0 1 1 0 1 2  | 0 2 3 0 5 12 22 0 0 0 0 0 
0 0 2 0 1 2  | 0 0 6 0 5 12 23 0 0 0 0 0 
0 0 1 1 1 2  | 0 0 3 4 5 12 24 0 0 15 0 0 
0 0 1 0 2 2  | 0 0 3 0 10 12 25 0 0 0 0 0 
0 0 1 0 1 3  | 0 0 3 0 5 18 26 0 0 0 0 0 
1 0 1 0 0 3  | 1 0 3 0 0 18 22 0 0 0 0 0 
0 1 1 0 0 3  | 0 2 3 0 0 18 23 0 0 0 0 0 
0 0 2 0 0 3  | 0 0 6 0 0 18 24 0 24 0 0 0 
0 0 1 1 0 3  | 0 0 3 4 0 18 25 0 0 0 0 0 
0 0 1 0 1 3  | 0 0 3 0 5 18 26 0 0 0 0 0 
0 0 1 0 0 4  | 0 0 3 0 0 24 27 27 0 0 0 0 
3 0 0 1 0 1  | 3 0 0 4 0 6 13 0 0 0 0 0 
2 1 0 1 0 1  | 2 2 0 4 0 6 14 0 0 0 0 0 
2 0 1 1 0 1  | 2 0 3 4 0 6 15 0 0 0 0 0 
2 0 0 2 0 1  | 2 0 0 8 0 6 16 0 0 0 0 0 
2 0 0 1 1 1  | 2 0 0 4 5 6 17 0 0 0 0 0 
2 0 0 1 0 2  | 2 0 0 4 0 12 18 0 0 0 0 0 
2 1 0 1 0 1  | 2 2 0 4 0 6 14 0 0 0 0 0 
1 2 0 1 0 1  | 1 4 0 4 0 6 15 0 0 0 0 0 
1 1 1 1 0 1  | 1 2 3 4 0 6 16 0 0 15 0 0 
1 1 0 2 0 1  | 1 2 0 8 0 6 17 0 0 0 0 0 
1 1 0 1 1 1  | 1 2 0 4 5 6 18 0 0 0 0 0 
1 1 0 1 0 2  | 1 2 0 4 0 12 19 0 0 0 0 

In [5]:
PRIORITY = 3

def train(net, train_iter,criterion, num_epochs, device,lr=None, weight_decay = 0):

    print('training on', device)
    net.to(device)
    #optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay = weight_decay)
    optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay = weight_decay)
    
    for epoch in range(num_epochs):
        #train_l_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        #train_loss_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        train_l_sum = 0.0
        train_loss_sum = 0.0
        
        
        n, start = 0, time.time()

        for x, y in train_iter:
            net.train()
            optimizer.zero_grad()
            x,y = x.to(device),y.to(device)
            
            y_hat = net(x)
            loss = criterion(y[:7], y_hat[:7]) + PRIORITY * criterion(y[7:],y_hat[7:])
            #loss = criterion(torch.atanh(y), torch.atanh(y_hat))
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_l_sum += loss.float()
                train_loss_sum += sum([abs(unpack(d)) for d in (y-y_hat)])
                n += 1


        print('epoch %d, loss %.4f, train loss %.3f, time %.1f sec, n=%d' %
              (epoch + 1, train_l_sum/n*SCALE, train_loss_sum/n, time.time() - start, n) )
    return

In [ ]:
criterion = nn.MSELoss(reduction = 'sum')
train(net, train_data, criterion,1000, device, 0.001)

training on cuda:0
epoch 1, loss 2.5814, train loss 6.515, time 17.7 sec, n=7776
epoch 2, loss 2.5792, train loss 6.519, time 15.2 sec, n=7776
epoch 3, loss 2.5760, train loss 6.514, time 15.2 sec, n=7776
epoch 4, loss 2.5726, train loss 6.509, time 14.5 sec, n=7776
epoch 5, loss 2.5693, train loss 6.504, time 14.4 sec, n=7776
epoch 6, loss 2.5659, train loss 6.497, time 14.1 sec, n=7776
epoch 7, loss 2.5626, train loss 6.490, time 14.2 sec, n=7776
epoch 8, loss 2.5593, train loss 6.486, time 14.2 sec, n=7776
epoch 9, loss 2.5561, train loss 6.486, time 14.8 sec, n=7776
epoch 10, loss 2.5528, train loss 6.480, time 15.3 sec, n=7776
epoch 11, loss 2.5496, train loss 6.479, time 14.4 sec, n=7776
epoch 12, loss 2.5463, train loss 6.472, time 14.5 sec, n=7776
epoch 13, loss 2.5431, train loss 6.470, time 14.4 sec, n=7776
epoch 14, loss 2.5399, train loss 6.465, time 14.4 sec, n=7776
epoch 15, loss 2.5368, train loss 6.460, time 14.2 sec, n=7776
epoch 16, loss 2.5337, train loss 6.455, time

In [ ]:
torch.save(net.state_dict(), './data/HM_learn_scores_' + str(2))

In [ ]:
net.load_state_dict(torch.load('./data/HM_learn_scores_1'))

In [ ]:
corr = 0

for x,y in train_data:
    net.eval()
    x,y = x.to(device),y.to(device)
    y_hat = net(x)

    hy = y.clone().detach()
    hyh = y_hat.clone().detach()
    
    dh = (hy-hyh)
    
    flag = False

    
    for i in range(OUTPUT_SIZE):
        if not unpack(dh[i]) == 0:
            flag = True
            break
    
    if not flag:
        corr += 1
        continue
    
    cont = ""
    for a in x:
        cont += str(int(a)) + " "
    cont += " | "
    for d in hy-hyh:
    #for d in hyh:
        cont += str(unpack(d)) + " "
    print(cont)
        
print("{0}".format(corr))


# DQN

In [ ]:
BATCH_SIZE = 128
MEMORY_SIZE = 10000
GAMMA = 0.8
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 500
TARGET_UPDATE = 5
ALPHA = 0.1
REWARD_SCALE = 1.0/50
TRAINSET_PERIOD = 1000

D_INPUT_SIZE = 18
D_OUTPUT_SIZE = 12

class D_Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(D_INPUT_SIZE)

policy_net = torch.nn.Sequential(
    D_Reshape(),
    nn.Linear(in_features=D_INPUT_SIZE, out_features=300),
    nn.LeakyReLU(),
    nn.Linear(300,100),
    nn.LeakyReLU(),
    nn.Linear(100,50)
    nn.LeakyReLU(),
    nn.Linear(50,OUTPUT_SIZE)
)

policy_net.apply(init_weights)
target_net = copy.deepcopy(policy_net)
target_net.eval()




In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """transition 저장"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    
    
memory = ReplayMemory(MEMORY_SIZE)